# Generate response

# Question answering using a search API and re-ranking

Searching for relevant information can sometimes feel like looking for a needle in a haystack, but don’t despair, GPTs can actually do a lot of this work for us. In this guide we explore a way to augment existing search systems with various AI techniques, helping us sift through the noise.

Two ways of retrieving information for GPT are:

1. **Mimicking Human Browsing:** [GPT triggers a search](https://openai.com/blog/chatgpt-plugins#browsing), evaluates the results, and modifies the search query if necessary. It can also follow up on specific search results to form a chain of thought, much like a human user would do.
2. **Retrieval with Embeddings:** Calculate [embeddings](https://platform.openai.com/docs/guides/embeddings) for your content and a user query, and then [retrieve the content](Question_answering_using_embeddings.ipynb) most related as measured by cosine similarity. This technique is [used heavily](https://blog.google/products/search/search-language-understanding-bert/) by search engines like Google.

These approaches are both promising, but each has their shortcomings: the first one can be slow due to its iterative nature and the second one requires embedding your entire knowledge base in advance, continuously embedding new content and maintaining a vector database.

By combining these approaches, and drawing inspiration from [re-ranking](https://www.sbert.net/examples/applications/retrieve_rerank/README.html) methods, we identify an approach that sits in the middle. **This approach can be implemented on top of any existing search system, like the Slack search API, or an internal ElasticSearch instance with private data**. Here’s how it works:

![search_augmented_by_query_generation_and_embeddings_reranking.png](../images/search_rerank_answer.png)

**Step 1: Search**

1.  User asks a question.
2.  GPT generates a list of potential queries.
3.  Search queries are executed in parallel.

**Step 2: Re-rank**

1.  Embeddings for each result are used to calculate semantic similarity to a generated hypothetical ideal answer to the user question.
2.  Results are ranked and filtered based on this similarity metric.

**Step 3: Answer**

1.  Given the top search results, the model generates an answer to the user’s question, including references and links.

This hybrid approach offers relatively low latency and can be integrated into any existing search endpoint, without requiring the upkeep of a vector database. Let's dive into it! We will use the [News API](https://newsapi.org/) as an example domain to search over.

## Setup

In addition to your `OPENAI_API_KEY`, you'll have to include a `NEWS_API_KEY` in your environment. You can get an API key [here](https://newsapi.org/).


In [9]:
# Dependencies
from datetime import date, timedelta  # date handling for fetching recent news
from IPython import display  # for pretty printing
import json  # for parsing the JSON api responses and model outputs
from numpy import dot  # for cosine similarity
import openai  # for using GPT and getting embeddings
import os  # for loading environment variables
import requests  # for making the API requests
from tqdm.notebook import tqdm  # for printing progress bars

# Load environment variables


GPT_MODEL = "gpt-3.5-turbo"


# Helper functions
def json_gpt(input: str):
    completion = openai.ChatCompletion.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "Output only valid JSON"},
            {"role": "user", "content": input},
        ],
        temperature=0.5,
    )

    text = completion.choices[0].message.content
    parsed = json.loads(text)

    return parsed


def embeddings(input): 
    response = openai.Embedding.create(model="text-embedding-ada-002", input=input)
    return [data.embedding for data in response.data]
from dotenv import load_dotenv
load_dotenv()
import os
openai.api_key = os.getenv("OPENAI_API_KEY")
news_api_key = os.getenv("NEWS_API_KEY")


## 1. Search

It all starts with a user question.


In [10]:
# User asks a question
# load the question
file_path = "data/final_transcript.txt"
with open(file_path, "r") as f:
    final_transcript = f.read()
USER_QUESTION = final_transcript
print(USER_QUESTION)

What factors make Vietnam an ideal destination for a visit?


Now, in order to be as exhaustive as possible, we use the model to generate a list of diverse queries based on this question.


In [11]:
QUERIES_INPUT = f"""
You have access to a search API that returns recent news articles.
Generate an array of search queries that are relevant to this question.
Use a variation of related keywords for the queries, trying to be as general as possible.
Include as many queries as you can think of, including and excluding terms.
For example, include queries like ['keyword_1 keyword_2', 'keyword_1', 'keyword_2'].
Be creative. The more queries you include, the more likely you are to find relevant results.

User question: {USER_QUESTION}

Format: {{"queries": ["query_1", "query_2", "query_3"]}}
"""

queries = json_gpt(QUERIES_INPUT)["queries"]

# Let's include the original question as well for good measure
queries.append(USER_QUESTION)

queries

['factors that make Vietnam an ideal destination for a visit',
 'reasons to visit Vietnam',
 'why Vietnam is a great place to visit',
 'advantages of visiting Vietnam',
 'benefits of traveling to Vietnam',
 'what makes Vietnam a must-visit destination',
 'things that attract tourists to Vietnam',
 'what sets Vietnam apart as a travel destination',
 'why you should consider visiting Vietnam',
 'top reasons to choose Vietnam for a vacation',
 'what makes Vietnam special for travelers',
 'unique aspects of Vietnam that make it an ideal destination',
 'why Vietnam should be on your travel bucket list',
 'What factors make Vietnam an ideal destination for a visit?']

The queries look good, so let's run the searches.


In [12]:
def search_news(
    query: str,
    news_api_key: str = news_api_key,
    num_articles: int = 5,
    from_datetime: str = "2023-09-12",  # the 2023 NBA finals were played in June 2023
    to_datetime: str = "2023-10-11",
) -> dict:
    response = requests.get(
        "https://newsapi.org/v2/everything",
        params={
            "q": query,
            "apiKey": news_api_key,
            "pageSize": num_articles,
            "sortBy": "relevancy",
            "from": from_datetime,
            "to": to_datetime,
        },
    )

    return response.json()


articles = []

for query in tqdm(queries):
    if len(query) > 499:
        continue
    result = search_news(query)
    if result["status"] == "ok":
        articles = articles + result["articles"]
    else:
        raise Exception(result["message"])

# remove duplicates
articles = list({article["url"]: article for article in articles}.values())

print("Total number of articles:", len(articles))
print("Top 5 articles of query 1:", "\n")

for article in articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print()


  0%|          | 0/14 [00:00<?, ?it/s]

Total number of articles: 90
Top 5 articles of query 1: 

Title: Beijing Chagrined As U.S. And Vietnam Draw Closer.
Description: Biden courts Hanoi in Washington’s battle with Beijing.  Vietnam is no ally, but it is cooperating for its own reasons.
Content: US President Joe Biden attends a welcoming ceremony hosted by Vietnam's Communist Party General ... ...

Title: A Tourist’s Guide to Love: Where to Watch & Stream Online
Description: A Tourist’s Guide to Love tells the story of Amanda Riley, an uptight travel executive whose life is turned upside down when her long-time boyfriend decides to end their relationship. Left broken after losing the love of her life, Amanda decides to visit Viet…
Content: A Tourists Guide to Love tells the story of Amanda Riley, an uptight travel executive whose life is ...

Title: North Korea’s Kim Jong Un Arrives In Russia For Putin Summit
Description: Kim, who traveled to Russia by armored train, is expected to discuss an arms deal with Moscow for its w

As we can see, oftentimes, the search queries will return a large number of results, many of which are not relevant to the original question asked by the user. In order to improve the quality of the final answer, we use embeddings to re-rank and filter the results.

## 2. Re-rank

Drawing inspiration from [HyDE (Gao et al.)](https://arxiv.org/abs/2212.10496), we first generate a hypothetical ideal answer to rerank our compare our results against. This helps prioritize results that look like good answers, rather than those similar to our question. Here’s the prompt we use to generate our hypothetical answer.


In [13]:
HA_INPUT = f"""
Generate a hypothetical answer to the user's question. This answer will be used to rank search results. 
Pretend you have all the information you need to answer, but don't use any actual facts. Instead, use placeholders
like NAME did something, or NAME said something at PLACE. 

User question: {USER_QUESTION}

Format: {{"hypotheticalAnswer": "hypothetical answer text"}}
"""

hypothetical_answer = json_gpt(HA_INPUT)["hypotheticalAnswer"]

hypothetical_answer


'Vietnam is an ideal destination for a visit due to its rich history and cultural heritage. The country offers breathtaking landscapes, including stunning beaches, majestic mountains, and lush green rice fields. Famous landmarks such as NAME Temple and NAME Pagoda attract tourists from all over the world. Additionally, the delicious cuisine, known for its bold flavors and fresh ingredients, is a highlight of any trip to Vietnam. NAME, a renowned travel blogger, said that exploring the bustling streets of Hanoi and experiencing the vibrant nightlife in Ho Chi Minh City are must-do activities. Overall, Vietnam offers a unique blend of history, natural beauty, and culinary delights, making it a perfect destination for travelers seeking an unforgettable experience.'

Now, let's generate embeddings for the search results and the hypothetical answer. We then calculate the cosine distance between these embeddings, giving us a semantic similarity metric. Note that we can simply calculate the dot product in lieu of doing a full cosine similarity calculation since the OpenAI embeddings are returned normalized in our API.


In [14]:
hypothetical_answer_embedding = embeddings(hypothetical_answer)[0]
article_embeddings = embeddings(
    [
        f"{article['title']} {article['description']} {article['content'][0:100]}"
        for article in articles
    ]
)

# Calculate cosine similarity
cosine_similarities = []
for article_embedding in article_embeddings:
    cosine_similarities.append(dot(hypothetical_answer_embedding, article_embedding))

cosine_similarities[0:10]


[0.7896581313500436,
 0.7608656435763905,
 0.7256594750596086,
 0.7848462334376771,
 0.7885668112043219,
 0.697278747846783,
 0.6904954335834975,
 0.7300791971384132,
 0.653784992645879,
 0.6709312284444438]

Finally, we use these similarity scores to sort and filter the results.


In [15]:
scored_articles = zip(articles, cosine_similarities)

# Sort articles by cosine similarity
sorted_articles = sorted(scored_articles, key=lambda x: x[1], reverse=True)

# Print top 5 articles
print("Top 5 articles:", "\n")

for article, score in sorted_articles[0:5]:
    print("Title:", article["title"])
    print("Description:", article["description"])
    print("Content:", article["content"][0:100] + "...")
    print("Score:", score)
    print()


Top 5 articles: 

Title: Vietnam’s appeal proves strong for Korean visitors
Description: Hanoi (VNA) – Attractive theme parks, stunning paradise-like beaches, and the highest peak in Indochina are just some of the reasons why more and more Korean tourists are flocking to Vietnam, accounting for up to 30% of the total foreign arrivals in the first…
Content: More and more Korean tourists are flocking to Vietnam
Hanoi (VNA) Attractive theme parks, stunning ...
Score: 0.891133554372068

Title: Tourism promotion takes flight in south for peak season
Description: HCM City (VNA) – The tourism industry, accommodation establishments, and travel firms are focusing on innovating products and services and strengthening promotion activities to popularise Vietnam as the leading tourist destination in Southeast Asia and to con…
Content: HCM City (VNA) The tourism industry, accommodation establishments, and travel firms are focusing on ...
Score: 0.8707824494296448

Title: Three most beautiful beaches

Awesome! These results look a lot more relevant to our original query. Now, let's use the top 5 results to generate a final answer.

## 3. Answer


In [21]:
formatted_top_results = [
    {
        "title": article["title"],
        "description": article["description"],
        "url": article["url"],
    }
    for article, _score in sorted_articles[0:5]
]

ANSWER_INPUT = f"""
Generate an answer to the user's question based on the given search results. 
TOP_RESULTS: {formatted_top_results}
USER_QUESTION: {USER_QUESTION}

Include as much information as possible in the answer. Reference the relevant search result urls as markdown links.
"""

completion = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": ANSWER_INPUT}],
    temperature=0.5,
    stream=True,
)

text = ""
for chunk in completion:
    text += chunk.choices[0].delta.get("content", "")
    display.clear_output(wait=True)
    display.display(display.Markdown(text))

Vietnam is becoming an increasingly popular destination for tourists, especially for Korean visitors. One reason for this is the country's attractive theme parks, stunning beaches, and the highest peak in Indochina. According to the search results, up to 30% of total foreign arrivals in Vietnam are Korean tourists. This article from Vietnam Plus provides more information on Vietnam's appeal to Korean visitors: [Vietnam’s appeal proves strong for Korean visitors](https://en.vietnamplus.vn/vietnams-appeal-proves-strong-for-korean-visitors/268833.vnp)

Additionally, Vietnam is actively promoting its tourism industry and improving its products and services to establish itself as the leading tourist destination in Southeast Asia. The tourism industry, accommodation establishments, and travel firms are focusing on innovation and promotion activities. This article from Vietnam Plus highlights the efforts being made to promote tourism in Vietnam: [Tourism promotion takes flight in south for peak season](https://en.vietnamplus.vn/tourism-promotion-takes-flight-in-south-for-peak-season/267928.vnp)

One of the top attractions in Vietnam is Phu Quoc island, which offers paradise-like beaches. Located off the coast of southern Vietnam in the Gulf of Thailand, Phu Quoc is considered a tropical paradise. This article from Vietnam Plus provides information on the three most beautiful beaches on Phu Quoc island: [Three most beautiful beaches on Phu Quoc island](https://en.vietnamplus.vn/three-most-beautiful-beaches-on-phu-quoc-island/267788.vnp)

Overall, Vietnam offers a diverse range of attractions and experiences, making it an ideal destination for a visit. For more information on the best travel destinations in the future, you can refer to this Forbes article: [You’ve Got To See This! The 24 Best Places To Travel In 2024](https://www.forbes.com/sites/christopherelliott/2023/10/01/youve-got-to-see-this-the-24-best-places-to-travel-in-2024/)

To enhance the shopping experience, Lotte Mall West Lake, Hanoi's first mega commercial complex, offers an exclusive shopping experience for Vietnamese customers. You can find more information about this complex in this article from VnExpress International: [Lotte Mall West Lake: Hanoi's first mega commercial complex](https://e.vnexpress.net/news/business/lotte-mall-west-lake-hanoi-s-first-mega-commercial-complex-4658902.html)